In [1]:
import cv2
import numpy as np
import os 
import pandas as pd


#### Generating selfie training data using webcam

In [105]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

name = input("Enter the file name:")
skip = 0
face_data = []
dataset_path = "./face_data/"
count = 1
while True:
    ret,frame = cap.read()
    if(ret==False): continue
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    faces = sorted(faces,key = lambda f:f[2]*f[3],reverse=True)
    for face in faces:
        x,y,w,h  = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        offset = 10;
        face_section = frame[y-offset:y+offset+h,x-offset:x+offset+w]
        face_section = cv2.resize(face_section,(100,100))
        cv2.imshow("face",face_section)
        if(skip%10==0):
            face_data.append(face_section)
            print(count)
            count +=1
        skip = skip+1        
    cv2.imshow("Frame",frame)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'): break
        
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

np.save(dataset_path+name+".npy",face_data,)
print("Data stored at :"+dataset_path+name+".npy")

cap.release()
cv2.destroyAllWindows()


Enter the file name:Harshu
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
(22, 30000)
Data stored at :./face_data/Harshu.npy


### Building Face Classifier

##### knn function 

In [2]:
def dis(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(X,Y,queryPoint,k=5):
    vals = []
    m = X.shape[0]
    for i in range(m):
        d =dis(queryPoint,X[i]);
        vals.append((d,Y[i]))
    vals = sorted(vals)
    vals = vals[:k]
    vals = np.array(vals)
    vals = vals[:,1]
    new_vals = np.unique(vals,return_counts=True) 
#     print(vals)
    max_freq_index = new_vals[1].argmax()
    return new_vals[0][max_freq_index]
#     print(vals)
    

##### Preparing data for knn from the face_data folder as X_train,Y_train

In [4]:
face_data = []
dataset_path = "./face_data/"
label = []
class_id = 0 #labels for given file
names = {}  #Mapping btw id-name
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx)
        print(data_item.shape)
        face_data.append(data_item)
        names[class_id] = fx[:-4]
        target = class_id*np.ones((data_item.shape[0],))
        class_id +=1 
        label.append(target)
        train_X = np.concatenate(face_data,axis=0)
        train_Y = np.concatenate(label,axis=0)
        print(train_X.shape)
        print(train_Y.shape)
        

Loaded Harshu.npy
(22, 30000)
(22, 30000)
(22,)
Loaded mamta.npy
(21, 30000)
(43, 30000)
(43,)
Loaded tilak.npy
(21, 30000)
(64, 30000)
(64,)
Loaded tilakparth.npy
(15, 30000)
(79, 30000)
(79,)


# Displaying names of the person 

In [5]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip = 0

while True:
    ret,frame = cap.read()
    if(ret==False): continue
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h  = face
        
        offset = 10;
        face_section = frame[y-offset:y+offset+h,x-offset:x+offset+w]
        face_section = cv2.resize(face_section,(100,100))
        out = knn(train_X,train_Y,face_section.flatten())
        cv2.rectangle(frame,(x,y+10),(x+w,y+h),(0,0,255),2)
        cv2.putText(frame,names[out],(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
# #         print(names[out])
#         if(skip%10==0):
#             print(names[out])
#         skip = skip+1        
    cv2.imshow("Frame",frame)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'): break


cap.release()
cv2.destroyAllWindows()  


